# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [5]:
class Encode:     
    def __init__(self, fileobj):
        self.fileobj = fileobj
    def read(self, size):
        return self.fileobj.read(size).encode()
    def readline(self, size=-1):
        return self.fileobj.readline(size).encode()

def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    with open(filename, 'r')as f:
        data = pickle.load(Encode(f))
    
    target = [i] * len(data)
    return data, target


In [6]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i,_ in enumerate(classes):       #romain et les trois lignes du for aussi
    ind_data, ind_target = load_pose_data(i)
    all_data.extend(ind_data)
    all_target.extend(ind_target)
    
print('total number of data', len(all_data))
print('total number of target', len(all_target))
all_data = np.asarray(all_data)
all_target = np.asarray(all_target)

total number of data 222
total number of target 222


In [7]:
#shuffle data#romain change les noms permutation_index_list et training_indices
permutation = np.random.permutation(len(all_data)) 
n_training_data = int(len(all_data) * 0.7)    
training_data = permutation[:n_training_data] 

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [8]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [9]:
# YOUR CODE HERE 
#romain change les noms learning_sample, learning_target
to_learn = np.asarray(all_data)[training_indices]     
to_reach= np.asarray(all_target)[training_indices]  
clf.fit(to_learn, to_reach)                   
                                                            
print('number of learning samples', len(to_learn))

number of learning samples 155


### predicting

In [10]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [11]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [12]:
expected = []
predicted = []

# YOUR CODE HERE
predicted = clf.predict(to_learn)
expected = to_reach

evaluate(expected, predicted)


Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        17
           2       0.96      1.00      0.98        24
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00        13
           7       1.00      0.88      0.93         8
           8       1.00      1.00      1.00        21
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00        28

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[18  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0  0]
 [ 0  0 24  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [13]:
expected = []
predicted = []
# YOUR CODE HERE                           
#romain vire toutes les lignes au dessus de ce comment et enlève mes commentaires aussi
predicted = clf.predict(all_data)   
expected = all_target              

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       0.97      1.00      0.98        30
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        26
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        52

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[23  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [14]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [15]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1]

(array([10]), 10)